In [47]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from scipy.io import loadmat
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [48]:
doa_data = loadmat('C:\Users\jnema\mat_files\DOA.mat')['DOA']
pressure_data = loadmat('C:\Users\jnema\mat_files\pressure.mat')['P']

In [49]:
n_points = 12
n_samples = 900  
time_interval = 60 / 48000  

for point in range(1, n_points + 1):
    time_values = [(i + 1) * time_interval for i in range(n_samples)]

    df = pd.DataFrame({
        'speaker': [point] * n_samples,
        'time': time_values
    })

    df.to_csv(f'dataset/point_{point}.csv', index=False, header=False)

print("CSV files created successfully.")

CSV files created successfully.


In [50]:
for point in range(1, n_points + 1):
    # Read the existing CSV file
    df = pd.read_csv(f'dataset/point_{point}.csv', header=None)

    doa_scaler = MinMaxScaler()
    pressure_scaler = MinMaxScaler()

    normalized_doa_data = doa_scaler.fit_transform(doa_data[point - 1][0])
    normalized_pressure_data = pressure_scaler.fit_transform(pressure_data[point - 1][0])

    normalized_doa_data_selected = normalized_doa_data[::500]
    normalized_pressure_data_selected = normalized_pressure_data[::500]
    
    combined_data = np.concatenate([normalized_doa_data_selected, normalized_pressure_data_selected], axis=1)


    df[['x', 'y', 'z', 'pressure']] = combined_data

    # Save the DataFrame back to the CSV file without a header
    df.to_csv(f'dataset/point_{point}.csv', index=False, header=False)

len(df)

900

In [51]:
n_points_to_concatenate = 12

# List to hold DataFrames
dfs = []

for point in range(1, n_points_to_concatenate + 1):
    # Read each CSV file
    df = pd.read_csv(f'dataset/point_{point}.csv', header=None, names=['point', 'time', 'x', 'y', 'z', 'pressure'])
    dfs.append(df)

# Concatenate all DataFrames
concatenated_df = pd.concat(dfs, ignore_index=True)

concatenated_df.describe()


,point,time,x,y,z,pressure
count,10800.000000,10800.000000,10800.000000,10800.000000,10800.000000,10800.000000
mean,6.500000,0.563125,0.682425,0.682321,0.711622,0.565566
std,3.452212,0.324774,0.210680,0.246109,0.183734,0.025682
min,1.000000,0.001250,0.000463,0.000014,0.001623,0.456896
25%,3.750000,0.282187,0.618343,0.602028,0.675558,0.546104
50%,6.500000,0.563125,0.758584,0.771828,0.771407,0.571330
75%,9.250000,0.844062,0.812151,0.839507,0.807540,0.582231
max,12.000000,1.125000,0.999555,0.999978,0.999972,0.644846


In [52]:
X = concatenated_df.loc[:8999,['point', 'time']]  # Features
y = concatenated_df.loc[:8999, ['x', 'y', 'z', 'pressure']]  # Target variable
X_test = concatenated_df.loc[9000:10800,['point', 'time']]  # Features
y_test = concatenated_df.loc[9000:10800, ['x', 'y', 'z', 'pressure']]  # Target variable

print(X.iloc[8999])
print(X_test.iloc[1799])

point    10.000
time      1.125
Name: 8999, dtype: float64
point    12.000
time      1.125
Name: 10799, dtype: float64


In [53]:
def create_sequences(X, y, time_steps):
    Xs, ys = [], []
    for i in range(int(len(X) / time_steps)):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i:(i + time_steps)].values)  # Ensure that ys is also a sequence
    return np.array(Xs), np.array(ys)

In [54]:
# Creating sequences
X, y = create_sequences(X, y, time_steps=900)
X_test, y_test = create_sequences(X_test, y_test, time_steps=900)
# Now X_train_seq has the shape (samples, time_steps, n_features)
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(10, 900, 2)
(10, 900, 4)
(2, 900, 2)
(2, 900, 4)


In [55]:
X_train, X_valid, y_train, y_valid = train_test_split(X, 
                                                  y, 
                                                  test_size= 0.2,
                                                  random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)

X_train shape: (8, 900, 2)
y_train shape: (8, 900, 4)
X_valid shape: (2, 900, 2)
y_valid shape: (2, 900, 4)


In [56]:
time_steps = 900
n_features = 2
l2_strength = 0.01
# Model Architecture
model = Sequential()
model.add(Dense(512, activation='sigmoid', input_shape=(time_steps, n_features), kernel_regularizer=l2(l2_strength)))

model.add(Dense(256, activation='sigmoid', kernel_regularizer=l2(l2_strength)))

model.add(Dense(128, activation='sigmoid', kernel_regularizer=l2(l2_strength)))

model.add(Dense(64, activation='sigmoid', kernel_regularizer=l2(l2_strength)))

model.add(Dense(32, activation='sigmoid', kernel_regularizer=l2(l2_strength)))

model.add(Dense(4))  # Output layer for 4 targets

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 900, 512)          1536      
                                                                 
 dense_17 (Dense)            (None, 900, 256)          131328    
                                                                 
 dense_18 (Dense)            (None, 900, 128)          32896     
                                                                 
 dense_19 (Dense)            (None, 900, 64)           8256      
                                                                 
 dense_20 (Dense)            (None, 900, 32)           2080      
                                                                 
 dense_21 (Dense)            (None, 900, 4)            132       
                                                                 
Total params: 176,228
Trainable params: 176,228
Non-tr

In [57]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
mcp_save = ModelCheckpoint('./saved/lstm_model.h5', save_best_only=True, monitor='val_accuracy', mode='auto', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)


In [58]:
# Training the Model
history = model.fit(X_train, y_train, epochs=50, batch_size=512, verbose=1, validation_data=[X_valid, y_valid], callbacks=[early_stopping, mcp_save, reduce_lr])

Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 6.9214 - accuracy: 0.3196
Epoch 1: val_accuracy improved from -inf to 0.32000, saving model to ./saved\lstm_model.h5
1/1 [==============================] - 1s 1s/step - loss: 6.9214 - accuracy: 0.3196 - val_loss: 6.6820 - val_accuracy: 0.3200 - lr: 0.0010
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 6.6820 - accuracy: 0.3196
Epoch 2: val_accuracy did not improve from 0.32000
1/1 [==============================] - 0s 128ms/step - loss: 6.6820 - accuracy: 0.3196 - val_loss: 6.4548 - val_accuracy: 0.3200 - lr: 0.0010
Epoch 3/50
1/1 [==============================] - ETA: 0s - loss: 6.4549 - accuracy: 0.3196
Epoch 3: val_accuracy did not improve from 0.32000
1/1 [==============================] - 0s 117ms/step - loss: 6.4549 - accuracy: 0.3196 - val_loss: 6.2393 - val_accuracy: 0.3200 - lr: 0.0010
Epoch 4/50
1/1 [==============================] - ETA: 0s - loss: 6.2393 - accuracy: 0.3196
Epoch 4: val_acc

In [59]:
model = load_model('./saved/lstm_model.h5')

In [60]:
predictions = model.predict(X_test)
print(type(predictions))
print(type(y_test))
print(predictions.shape)
print(y_test.shape)

1/1 [==============================] - 0s 207ms/step
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(2, 900, 4)
(2, 900, 4)


In [61]:
y_test = y_test.reshape(-1, 4)
predictions = predictions.reshape(-1, 4)

In [62]:
nan_indices_y_test = np.isnan(y_test)
nan_indices_predictions = np.isnan(predictions)

print("NaNs in y_test:", np.sum(nan_indices_y_test))
print("NaNs in predictions:", np.sum(nan_indices_predictions))

NaNs in y_test: 0
NaNs in predictions: 0


In [63]:
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error (MAE):", mae)

mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error (MSE):", mse)

print(y_test)
print(predictions)

Mean Absolute Error (MAE): 0.1416753081917788
Mean Squared Error (MSE): 0.037907353267914466
[[0.54329053 0.12816405 0.16846877 0.57439567]
 [0.41320724 0.8685161  0.82658195 0.58828469]
 [0.88018872 0.35012888 0.21184088 0.60793082]
 ...
 [0.78945837 0.79800852 0.77820459 0.57451206]
 [0.77132232 0.80557342 0.78810098 0.57452327]
 [0.60027901 0.75870822 0.08402798 0.55861756]]
[[0.6517604  0.71465945 0.7123596  0.6441049 ]
 [0.6517603  0.7146596  0.7123596  0.64410484]
 [0.6517604  0.7146595  0.7123597  0.64410496]
 ...
 [0.651757   0.71462816 0.7123835  0.6441062 ]
 [0.6517571  0.7146282  0.7123835  0.64410615]
 [0.65173864 0.7146301  0.7123978  0.64414644]]
